# AutoBuild
By: [Linxin Song](https://linxins97.github.io/), [Jieyu Zhang](https://jieyuz2.github.io/)
Reference: [Agent AutoBuild](https://microsoft.github.io/autogen/blog/2023/11/26/Agent-AutoBuild/)

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we introduce a new class, `AgentBuilder`, to help user build an automatic task solving process powered by multi-agent system. Specifically, in `build()`, we prompt a LLM to create multiple participant agent and initialize a group chat, and specify whether this task need programming to solve. AgentBuilder also support open-source LLMs by [vLLM](https://docs.vllm.ai/en/latest/index.html) and [Fastchat](https://github.com/lm-sys/FastChat). Check the supported model list [here](https://docs.vllm.ai/en/latest/models/supported_models.html).

## Requirement

AutoBuild require `pyautogen[autobuild]`, which can be installed by the following command:

In [ ]:
%pip install pyautogen[autobuild]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvid

## Step 1: prepare configuration and some useful functions
Prepare a `config_file_or_env` for assistant agent to limit the choice of LLM you want to use in this task. This config can be a path of json file or a name of environment variable. A `default_llm_config` is also required for initialize the specific config of LLMs like seed, temperature, etc...

In [ ]:
import autogen
from autogen.agentchat.contrib.agent_builder import AgentBuilder
from google.colab import userdata
import os
config_file_or_env = "./OAI_CONFIG_LIST.json"

llm_config = {"temperature": 0}
config_list=[
    {
        'model': userdata.get('AZURE_OPENAI_DEPLOYMENT'),
        'api_key': userdata.get('AZURE_OPENAI_API_KEY'),
        'base_url': userdata.get('AZURE_OPENAI_ENDPOINT'),
        'api_version': '2023-03-15-preview',
        'api_type': 'azure',

    }
]

os.environ['AZURE_OPENAI_DEPLOYMENT'] = userdata.get('AZURE_OPENAI_DEPLOYMENT')
os.environ['AZURE_OPENAI_API_KEY'] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ['AZURE_OPENAI_ENDPOINT'] = userdata.get('AZURE_OPENAI_ENDPOINT')
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')
#config_list = autogen.config_list_from_json(config_file_or_env, filter_dict={"model": ["gpt-4-turbo", "gpt-4"]})


def start_task(execution_task: str, agent_list: list, coding=True):
    group_chat = autogen.GroupChat(
        agents=agent_list,
        messages=[],
        max_round=20,
        allow_repeat_speaker=agent_list[:-1] if coding is True else agent_list,
    )
    manager = autogen.GroupChatManager(
        groupchat=group_chat,
        llm_config={"config_list": config_list, **llm_config},
    )
    agent_list[0].initiate_chat(manager, message=execution_task)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Step 2: create a AgentBuilder
Create a `AgentBuilder` with the specified `config_path_or_env`. AgentBuilder will use `gpt-4` in default to complete the whole process, you can specify the `builder_model` and `agent_model` to other OpenAI model to match your task.
You can also specify an open-source LLM supporting by vLLM and FastChat, see blog for more details.

In [ ]:
builder = AgentBuilder(
   config_file_or_env=config_file_or_env
)

## Step 3: specify a building task

Specify a building task with a general description. Building task will help build manager (a LLM) decide what agents should be built.

In [ ]:
building_task = "Generate agents that can conduct market research on how to diversify an investment of 1,00,000 rupees among stocks. The agents should gather and analyze relevant data from this website https://www.moneycontrol.com/ only to provide a detailed recommendation on how to allocate the investment across assets based on current market trends and forecasts.The agents should find the most recent data which is today's data from the given source website and use that data only."

## Step 4: build group chat agents
Use `build()` to let build manager (the specified `builder_model`) complete the group chat agents generation. If you think coding is necessary in your task, you can use `coding=True` to add a user proxy (an automatic code interpreter) into the agent list, like:
```python
builder.build(building_task, default_llm_config, coding=True)
```
If `coding` is not specified, AgentBuilder will determine on its own whether the user proxy should be added or not according to the task.

In [ ]:
agent_list, agent_configs = builder.build(building_task, llm_config)

==> Generating agents...
['MarketAnalysis_Expert', 'DataScience_Expert', 'StockResearch_Expert', 'FinancialTrends_Expert', 'InvestmentStrategy_Expert'] are generated.
==> Generating system message...
Preparing system message for MarketAnalysis_Expert
Preparing system message for DataScience_Expert
Preparing system message for StockResearch_Expert
Preparing system message for FinancialTrends_Expert
Preparing system message for InvestmentStrategy_Expert
==> Generating description...
Preparing description for MarketAnalysis_Expert
Preparing description for DataScience_Expert
Preparing description for StockResearch_Expert
Preparing description for FinancialTrends_Expert
Preparing description for InvestmentStrategy_Expert
==> Creating agents...
Creating agent MarketAnalysis_Expert...
Creating agent DataScience_Expert...
Creating agent StockResearch_Expert...
Creating agent FinancialTrends_Expert...
Creating agent InvestmentStrategy_Expert...
Adding user console proxy...


## Step 5: execute task
Let agents generated in `build()` to complete the task collaboratively in a group chat.

In [ ]:
start_task(
    execution_task="""Conduct market research to analyze how to diversify an investment of 1,00,000 rupees in stocks. Use only the Moneycontrol website to gather the necessary information. The website link is https://www.moneycontrol.com/. Gather recent data from the given source only, analyze market trends and forecasts, and provide a detailed recommendation on the optimal allocation of the investment across these assets. Ensure that the stock prices are today's prices and divide the investment strategically for maximum profit. Retrieve the accurate price of the stocks from the page. Perform up to a maximum of 5 rounds and present the final output in a table.""",
    agent_list=agent_list,
    coding=agent_configs["coding"],
)

MarketAnalysis_Expert (to chat_manager):

Conduct market research to analyze how to diversify an investment of 1,00,000 rupees in stocks. Use only the Moneycontrol website to gather the necessary information. The website link is https://www.moneycontrol.com/. Gather recent data from the given source only, analyze market trends and forecasts, and provide a detailed recommendation on the optimal allocation of the investment across these assets. Ensure that the stock prices are today's prices and divide the investment strategically for maximum profit. Retrieve the accurate price of the stocks from the page. Perform up to a maximum of 5 rounds and present the final output in a table.

--------------------------------------------------------------------------------

Next speaker: Computer_terminal

Computer_terminal (to chat_manager):

There is no code from the last 1 message for me to execute. Group chat manager should let other participants to continue the conversation. If the group chat 

## Step 6 (Optional): clear all agents and prepare for the next task
You can clear all agents generated in this task by the following code if your task is completed or the next task is largely different from the current task. If the agent's backbone is an open-source LLM, this process will also shut down the endpoint server. If necessary, you can use `recycle_endpoint=False` to retain the previous open-source LLMs' endpoint server.

In [ ]:
builder.clear_all_agents(recycle_endpoint=True)

All agents have been cleared.


In [ ]:
saved_path = builder.save()

Building config saved to ./save_config_7f07bc09833f32df2903f041daa87c96.json


After that, you can load the saved config and skip the building process. AgentBuilder will create agents with those information without prompting the builder manager.